In [1]:
import os
import numpy as np
import random
from random import sample
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import gzip
import cv2
from scipy.spatial import KDTree

In [2]:
"""将单个gz文件解压为txt文件"""
def unzip_file(input_file, output_file):
    gfile = gzip.GzipFile(input_file)
    open(output_file,"wb").write(gfile.read())
    gfile.close()

In [3]:
"""解压所有gz文件"""
def unzip_folder():
    idt_feature_zip_dir = "../视频合集/拔管/idt_feature"
    idt_feature_unzip_dir = "../视频合集/拔管/idt特征"
    for video_name in os.listdir(idt_feature_zip_dir):
        video_path = os.path.join(idt_feature_zip_dir, video_name)
        out_path = os.path.join(idt_feature_unzip_dir, video_name)
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        for clip_name in os.listdir(video_path):
            txt_name = clip_name.replace("gz","txt")
            input_file = os.path.join(video_path, clip_name)
            output_file = os.path.join(out_path,txt_name)
            unzip_file(input_file,output_file)

In [4]:
unzip_folder()

In [4]:
"""统计所有视频的idt向量，构建并训练bow词袋模型"""
def create_dictionary(features):
    dictionary_size=20
    bow = cv2.BOWKMeansTrainer(dictionary_size)
    for i, video in enumerate(features):
        for clip_idx, clip in enumerate(video):
            bow.add(np.float32(clip))
    dictionary = bow.cluster()
    return dictionary

In [5]:
"""重构词向量"""
def create_bow(dictionary, features, save_paths):
    kdtree = KDTree(dictionary)
    bins = dictionary.shape[0]
    for i, video in enumerate(features):
        for clip_idx, clip in enumerate(video):
            dis, indices = kdtree.query(clip)
            hist, _ = np.histogram(indices, bins=bins)
            hist = hist / len(indices)

            store_path = save_paths[i][clip_idx]  # 储存当前clip的词向量------特征
            np.save(store_path, hist)

In [6]:
"""主函数：处理视频并保存每个clip的bow向量"""
def idt_clip_bow():
    idt_feature_dir = "../视频合集/拔管/idt特征"
    save_dir = "../视频合集/拔管/idt特征bow"
    clips_save_path = []
    file_dir = os.listdir(idt_feature_dir)
    file_dir.sort()
    features = []
    for video_name in file_dir:
        video_path = os.path.join(idt_feature_dir, video_name)
        clips_path_list = os.listdir(video_path)
        clips_path_list.sort()
        clip_bow_path = os.path.join(save_dir, video_name)
        if not os.path.exists(clip_bow_path):
            os.makedirs(clip_bow_path)
        videos = []
        tmp = []
        for clip in clips_path_list:
            tmp.append(os.path.join(clip_bow_path, clip))
            clip_path = os.path.join(video_path, clip)
            data = np.loadtxt(clip_path)
            # 前10列为轨迹信息，30维轨迹，96维HOG，108维HOF，96维MBHX，96维MBHY
            feature_data = data[:, 40:436]
            videos.append(feature_data)
        features.append(videos)
        clips_save_path.append(tmp)
    print("Done: features 创建完成")
    # 构建字典
    dictionary = create_dictionary(features)
    print("Done: dictionary 创建完成")
    create_bow(dictionary, features, clips_save_path)
    print("Done: bow 创建完成")

In [ ]:
idt_clip_bow()

In [2]:
"""读取保存的bow词特征向量"""
def read_bow():
    bow_feature_path = "./idt特征bow"
    labels_path = "./标签信息"
    feature_sample = [[],[],[],[]]
    filename_sample = [[],[],[],[]]
    for videoname in sorted(os.listdir(bow_feature_path)):
        video_feature_path = os.path.join(bow_feature_path,videoname)   # 视频路径
        label_path = os.path.join(labels_path,videoname+".txt")
        # 读取标签文件
        label = []
        with open(label_path,"r",encoding='utf-8') as f:
            while True:
                line = f.readline()
                if not line:
                    break
                line = line.split('，')
                label.append(eval(line[1]))
        # 根据label读取每个clip的特征文件
        for i in range(len(label)):
            clipname = "clip"+str(i+1)+".txt.npy"
            tmp_path = videoname+"/"+clipname
            clip_feature_path = os.path.join(video_feature_path,clipname)
            feature = np.load(clip_feature_path)   # 读取feature文件
            feature_sample[label[i]-1].append(feature.tolist())
            filename_sample[label[i]-1].append(tmp_path)
    return feature_sample,filename_sample

In [23]:
"""划分数据集并利用svm，knn训练"""
def train_svm_knn(clf_num,method):
    feature_sample,filename_sample = read_bow()
    print("类别1样本数量：",len(feature_sample[0]),"类别1文件数量：",len(filename_sample[0]))
    print("类别2样本数量：",len(feature_sample[1]),"类别2文件数量：",len(filename_sample[1]))
    print("类别3样本数量：",len(feature_sample[2]),"类别3文件数量：",len(filename_sample[2]))
    print("类别4样本数量：",len(feature_sample[3]),"类别4文件数量：",len(filename_sample[3]))
    
    # 选择随机种子
    random.seed(0)
    if clf_num == 2:
        # 二分类
        feature_sample_array = sample(feature_sample[0],361)+feature_sample[1]+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],361))+[1]*len(feature_sample[1]+feature_sample[3])
        filename_sample_array = sample(filename_sample[0],361)+filename_sample[1]+filename_sample[3]
    elif clf_num == 3:
        # 三分类
        feature_sample_array = sample(feature_sample[0],159)+sample(feature_sample[1],159)+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],159))+[1]*len(sample(feature_sample[1],159))+[2]*len(feature_sample[3])
        filename_sample_array = sample(filename_sample[0],159)+sample(filename_sample[1],159)+filename_sample[3]
    elif clf_num == 4:
        # 四分类
        feature_sample_array = sample(feature_sample[0],104)+sample(feature_sample[1],104)+feature_sample[2]+sample(feature_sample[3],104)
        label_array = [0]*len(sample(feature_sample[0],104))+[1]*len(sample(feature_sample[1],104))+[2]*len(feature_sample[2])+[3]*len(sample(feature_sample[3],104))
        filename_sample_array = sample(filename_sample[0],104)+sample(filename_sample[1],104)+filename_sample[2]+sample(filename_sample[3],104)
    
    # 分层k折交叉验证
    skf = StratifiedKFold(n_splits=10)
    # 储存每一轮的结果
    train_score_list = []
    test_score_list = []
    precision_score_list = []
    recall_score_list = []
    f1_score_list = []
    feature_sample_array = preprocessing.scale(feature_sample_array)   # processing预处理
    for train, test in skf.split(feature_sample_array,label_array):
        train_data = np.array(feature_sample_array)[train,:]   # 划分好的训练集
        train_label = np.array(label_array)[train]
        test_data = np.array(feature_sample_array)[test,:]   # 划分好的测试集
        test_label = np.array(label_array)[test]
        if method == "SVM":
#             clf = svm.SVC(kernel='linear',decision_function_shape='ovr',max_iter=1e7)
            clf = svm.SVC(kernel='linear',decision_function_shape='ovr')
            clf.fit(train_data,train_label)
        if method == "KNN":
            clf = KNeighborsClassifier(n_neighbors=10)
            clf.fit(train_data,train_label)
        
        # 采用混淆矩阵计算各种评价指标
        predicted_train_label = clf.predict(train_data)
        predicted_test_label = clf.predict(test_data)
#         print("当前折训练集准确率：",metrics.accuracy_score(train_label,predicted_train_label))
        train_score_list.append(metrics.accuracy_score(train_label,predicted_train_label))
#         print("当前折测试集准确率：",metrics.accuracy_score(test_label,predicted_test_label))
        test_score_list.append(metrics.accuracy_score(test_label,predicted_test_label))
#         print("当前折精准值：",metrics.precision_score(test_label,predicted_test_label,average='weighted'))
        precision_score_list.append(metrics.precision_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折召回率：",metrics.recall_score(test_label,predicted_test_label,average='weighted'))
        recall_score_list.append(metrics.recall_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折F1：",metrics.f1_score(test_label,predicted_test_label,average='weighted'))
        f1_score_list.append(metrics.f1_score(test_label,predicted_test_label,average='weighted'))
#         print("Done!*****************************************************")
       
    print("Method:",method," Clf_num",clf_num)
    print("训练集准确率：",np.array(train_score_list).mean())
    print("测试集准确率：",np.array(test_score_list).mean())
    print("精确率：",np.array(precision_score_list).mean())
    print("召回率：",np.array(recall_score_list).mean())
    print("F1得分：",np.array(f1_score_list).mean())

In [24]:
"""进行2，3，4分类，并利用svm，knn为分类器做对比实验"""
cls_num = [2,3,4]
methods = ["SVM","KNN"]
for num in cls_num:
    for method in methods:
        train_svm_knn(num,method)

类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: SVM  Clf_num 2
训练集准确率： 0.7520789380111415
测试集准确率： 0.6953957382039574
精确率： 0.7061062112921122
召回率： 0.6953957382039574
F1得分： 0.6925300198892972
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: KNN  Clf_num 2
训练集准确率： 0.7900884200545217
测试集准确率： 0.6898021308980213
精确率： 0.6997011278507989
召回率： 0.6898021308980213
F1得分： 0.6850316482870648
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: SVM  Clf_num 3
训练集准确率： 0.5874624600205995
测试集准确率： 0.4379432624113475
精确率： 0.4243153188234497
召回率： 0.4379432624113475
F1得分： 0.4188351937432622
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Method: KNN  Clf_num 3
训练集准确率： 0.623568601940695
测试集准确率： 0.486081560283688
精确率： 0.47287501217068073
召回率： 0.486081560283688
F1得分： 0.461

In [16]:
"""利用mlp、rvm、random forest分类器进行分类实验"""
"""补充实验"""
def train_supplement(clf_num,method):
    feature_sample,filename_sample = read_bow()
    print("类别1样本数量：",len(feature_sample[0]),"类别1文件数量：",len(filename_sample[0]))
    print("类别2样本数量：",len(feature_sample[1]),"类别2文件数量：",len(filename_sample[1]))
    print("类别3样本数量：",len(feature_sample[2]),"类别3文件数量：",len(filename_sample[2]))
    print("类别4样本数量：",len(feature_sample[3]),"类别4文件数量：",len(filename_sample[3]))
    
    # 选择随机种子
    random.seed(0)
    if clf_num == 2:
        # 二分类
        feature_sample_array = sample(feature_sample[0],361)+feature_sample[1]+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],361))+[1]*len(feature_sample[1]+feature_sample[3])
        filename_sample_array = sample(filename_sample[0],361)+filename_sample[1]+filename_sample[3]
    elif clf_num == 3:
        # 三分类
        feature_sample_array = sample(feature_sample[0],159)+sample(feature_sample[1],159)+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],159))+[1]*len(sample(feature_sample[1],159))+[2]*len(feature_sample[3])
        filename_sample_array = sample(filename_sample[0],159)+sample(filename_sample[1],159)+filename_sample[3]
    elif clf_num == 4:
        # 四分类
        feature_sample_array = sample(feature_sample[0],104)+sample(feature_sample[1],104)+feature_sample[2]+sample(feature_sample[3],104)
        label_array = [0]*len(sample(feature_sample[0],104))+[1]*len(sample(feature_sample[1],104))+[2]*len(feature_sample[2])+[3]*len(sample(feature_sample[3],104))
        filename_sample_array = sample(filename_sample[0],104)+sample(filename_sample[1],104)+filename_sample[2]+sample(filename_sample[3],104)
    
    # 分层k折交叉验证
    skf = StratifiedKFold(n_splits=10)
    # 储存每一轮的结果
    train_score_list = []
    test_score_list = []
    precision_score_list = []
    recall_score_list = []
    f1_score_list = []
    feature_sample_array = preprocessing.scale(feature_sample_array)   # processing预处理
    for train, test in skf.split(feature_sample_array,label_array):
        train_data = np.array(feature_sample_array)[train,:]   # 划分好的训练集
        train_label = np.array(label_array)[train]
        test_data = np.array(feature_sample_array)[test,:]   # 划分好的测试集
        test_label = np.array(label_array)[test]
        if method == "MLP":
            # 利用mlp进行多分类
            from sklearn.neural_network import MLPClassifier
#             clf = MLPClassifier(solver='lbfgs',alpha=1e-3,hidden_layer_sizes=(64,3))
#             clf = MLPClassifier(solver='lbfgs',alpha=1e-6,hidden_layer_sizes=(128,6))
            clf = MLPClassifier(solver='lbfgs',alpha=1e-6,hidden_layer_sizes=(256,9))
            clf.fit(train_data,train_label)
        if method == "RVM":
            from sklearn_rvm import EMRVC
            clf = EMRVC(kernel='rbf',max_iter=10)
            clf.fit(train_data,train_label)
        if method == "RandomForest":
            from sklearn.ensemble import RandomForestClassifier
            clf = RandomForestClassifier(max_depth=2)
            clf.fit(train_data,train_label)
        # 采用混淆矩阵计算各种评价指标
        predicted_train_label = clf.predict(train_data)
        predicted_test_label = clf.predict(test_data)
        print("当前折训练集准确率：",metrics.accuracy_score(train_label,predicted_train_label))
        train_score_list.append(metrics.accuracy_score(train_label,predicted_train_label))
        print("当前折测试集准确率：",metrics.accuracy_score(test_label,predicted_test_label))
        test_score_list.append(metrics.accuracy_score(test_label,predicted_test_label))
#         print("当前折精准值：",metrics.precision_score(test_label,predicted_test_label,average='weighted'))
        precision_score_list.append(metrics.precision_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折召回率：",metrics.recall_score(test_label,predicted_test_label,average='weighted'))
        recall_score_list.append(metrics.recall_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折F1：",metrics.f1_score(test_label,predicted_test_label,average='weighted'))
        f1_score_list.append(metrics.f1_score(test_label,predicted_test_label,average='weighted'))
#         print("Done!*****************************************************")
       
    print("Method:",method," Clf_num:",clf_num)
    print("训练集准确率：",np.array(train_score_list).mean())
    print("测试集准确率：",np.array(test_score_list).mean())
    print("精确率：",np.array(precision_score_list).mean())
    print("召回率：",np.array(recall_score_list).mean())
    print("F1得分：",np.array(f1_score_list).mean())

In [17]:
cls_num = [2,3,4]
methods = ["RVM"]
for num in cls_num:
    for method in methods:
        train_supplement(num,method)

类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "


当前折训练集准确率： 0.8228043143297381
当前折测试集准确率： 0.7945205479452054


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "


当前折训练集准确率： 0.901386748844376
当前折测试集准确率： 0.684931506849315


...

当前折训练集准确率： 0.936
当前折测试集准确率： 0.5121951219512195


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will c

当前折训练集准确率： 0.9226666666666666
当前折测试集准确率： 0.5609756097560976


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will c

当前折训练集准确率： 0.936
当前折测试集准确率： 0.5365853658536586
Method: RVM  Clf_num: 4
训练集准确率： 0.9260092691622104
测试集准确率： 0.511672473867596
精确率： 0.511203873797515
召回率： 0.511672473867596
F1得分： 0.5041998919311306


In [27]:
"""划分数据集并利用DT决策树训练"""
def train_dt(clf_num):
    feature_sample, filename_sample = read_bow()
    print("类别1样本数量：",len(feature_sample[0]),"类别1文件数量：",len(filename_sample[0]))
    print("类别2样本数量：",len(feature_sample[1]),"类别2文件数量：",len(filename_sample[1]))
    print("类别3样本数量：",len(feature_sample[2]),"类别3文件数量：",len(filename_sample[2]))
    print("类别4样本数量：",len(feature_sample[3]),"类别4文件数量：",len(filename_sample[3]))
    
    # 选择随机种子
    random.seed(0)
    if clf_num == 2:
        # 二分类
        feature_sample_array = sample(feature_sample[0],361)+feature_sample[1]+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],361))+[1]*len(feature_sample[1]+feature_sample[3])
        filename_sample_array = sample(filename_sample[0],361)+filename_sample[1]+filename_sample[3]
    elif clf_num == 3:
        # 三分类
        feature_sample_array = sample(feature_sample[0],159)+sample(feature_sample[1],159)+feature_sample[3]
        label_array = [0]*len(sample(feature_sample[0],159))+[1]*len(sample(feature_sample[1],159))+[2]*len(feature_sample[3])
        filename_sample_array = sample(filename_sample[0],159)+sample(filename_sample[1],159)+filename_sample[3]
    elif clf_num == 4:
        # 四分类
        feature_sample_array = sample(feature_sample[0],104)+sample(feature_sample[1],104)+feature_sample[2]+sample(feature_sample[3],104)
        label_array = [0]*len(sample(feature_sample[0],104))+[1]*len(sample(feature_sample[1],104))+[2]*len(feature_sample[2])+[3]*len(sample(feature_sample[3],104))
        filename_sample_array = sample(filename_sample[0],104)+sample(filename_sample[1],104)+filename_sample[2]+sample(filename_sample[3],104)
    
    # 分层k折交叉验证
    skf = StratifiedKFold(n_splits=10)
    # 储存每一轮的结果
    train_score_list = []
    test_score_list = []
    precision_score_list = []
    recall_score_list = []
    f1_score_list = []
    feature_sample_array = preprocessing.scale(feature_sample_array)   # processing预处理
    for train, test in skf.split(feature_sample_array,label_array):
        train_data = np.array(feature_sample_array)[train,:]   # 划分好的训练集
        train_label = np.array(label_array)[train]
        test_data = np.array(feature_sample_array)[test,:]   # 划分好的测试集
        test_label = np.array(label_array)[test]
        
        clf = DecisionTreeClassifier(random_state=0,max_depth=3)
        clf.fit(train_data,train_label)
        
        # 采用混淆矩阵计算各种评价指标
        predicted_train_label = clf.predict(train_data)
        predicted_test_label = clf.predict(test_data)
#         print("当前折训练集准确率：",metrics.accuracy_score(train_label,predicted_train_label))
        train_score_list.append(metrics.accuracy_score(train_label,predicted_train_label))
#         print("当前折测试集准确率：",metrics.accuracy_score(test_label,predicted_test_label))
        test_score_list.append(metrics.accuracy_score(test_label,predicted_test_label))
#         print("当前折精准值：",metrics.precision_score(test_label,predicted_test_label,average='weighted'))
        precision_score_list.append(metrics.precision_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折召回率：",metrics.recall_score(test_label,predicted_test_label,average='weighted'))
        recall_score_list.append(metrics.recall_score(test_label,predicted_test_label,average='weighted'))
#         print("当前折F1：",metrics.f1_score(test_label,predicted_test_label,average='weighted'))
        f1_score_list.append(metrics.f1_score(test_label,predicted_test_label,average='weighted'))
#         print("Done!*****************************************************")
       
    print("Clf_num: ",clf_num)
    print("训练集准确率：",np.array(train_score_list).mean())
    print("测试集准确率：",np.array(test_score_list).mean())
    print("精确率：",np.array(precision_score_list).mean())
    print("召回率：",np.array(recall_score_list).mean())
    print("F1得分：",np.array(f1_score_list).mean())

In [28]:
"""进行2，3，4分类，并利用dt为分类器做对比实验"""
cls_num = [2,3,4]
for num in cls_num:
    train_dt(num)

类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Clf_num:  2
训练集准确率： 0.7654649757022639
测试集准确率： 0.7007039573820395
精确率： 0.7094540395451617
召回率： 0.7007039573820395
F1得分： 0.6974011584342602
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Clf_num:  3
训练集准确率： 0.5886322979346236
测试集准确率： 0.45487588652482264
精确率： 0.41436686944965617
召回率： 0.45487588652482264
F1得分： 0.40201025761263126
类别1样本数量： 511 类别1文件数量： 511
类别2样本数量： 202 类别2文件数量： 202
类别3样本数量： 104 类别3文件数量： 104
类别4样本数量： 159 类别4文件数量： 159
Clf_num:  4
训练集准确率： 0.5998837789661319
测试集准确率： 0.518583042973287
精确率： 0.5425090272020141
召回率： 0.518583042973287
F1得分： 0.5019185454348859


C:\Users\admin\Anaconda3\envs\UEX\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.autograd import Variable
import random

"""分类器参数初始化"""
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
        
"""构建数据迭代器"""
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

"""训练过程"""
def train_process(net,updater,loss,features,labels,batch_size,num_epochs):
    data_iter = load_array((features,labels),batch_size)
    train_acc_list = []
    for epoch in range(num_epochs):
        running_corrects = 0
        for X,y in data_iter:
            X = Variable(X,requires_grad=True)
            y = Variable(y)
            outputs = net(X)
            l = loss(outputs,y)
            
            y_hat = F.softmax(outputs,dim=1).argmax(axis=1)
            label = y.argmax(axis=1)
            updater.zero_grad()
            l.mean().backward()
            updater.step()
            
            # 记录每隔batch的准确率
            running_corrects += torch.sum(y_hat==label.data)
        train_acc = running_corrects.double() / labels.shape[0]
#         print("epoch {}".format(epoch+1)," train_acc {}".format(train_acc))
        train_acc_list.append(train_acc)
    return train_acc_list
#     print("训练完成")

"""测试过程"""
def test_process(net,features,labels):
    y_hat = F.softmax(net(features),dim=1).argmax(axis=1)
    y = labels.argmax(axis=1)
    count = 0
    sample =0 
    for i, j in zip(y_hat,y):
        sample += 1
        if int(i) == int(j):
            count+=1
    accuracy = count / sample
    return list(y_hat), accuracy

In [16]:
def train_fc(clf_num):
    feature_sample = read_bow()
    print("类别1样本数量：",len(feature_sample[0]))
    print("类别2样本数量：",len(feature_sample[1]))
    print("类别3样本数量：",len(feature_sample[2]))
    print("类别4样本数量：",len(feature_sample[3]))
    # 对样本数据集进行k折划分
    # 选择随机种子
    random.seed(0)
    if clf_num == 2:
        # 二分类
        feature_sample_array = np.array(sample(feature_sample[0],361)+feature_sample[1]+feature_sample[3])
        label_array = np.array([0]*len(sample(feature_sample[0],361))+[1]*len(feature_sample[1]+feature_sample[3]))
    elif clf_num == 3:
        # 三分类
        feature_sample_array = np.array(sample(feature_sample[0],159)+sample(feature_sample[1],159)+feature_sample[3])
        label_array = np.array([0]*len(sample(feature_sample[0],159))+[1]*len(sample(feature_sample[1],159))+[2]*len(feature_sample[3]))
    elif clf_num == 4:
        # 四分类
        feature_sample_array = np.array(sample(feature_sample[0],104)+sample(feature_sample[1],104)+feature_sample[2]+sample(feature_sample[3],104))
        label_array = np.array([0]*len(sample(feature_sample[0],104))+[1]*len(sample(feature_sample[1],104))+[2]*len(feature_sample[2])+[3]*len(sample(feature_sample[3],104)))
    
#     feature_sample_array = feature_sample_array[:,0:2]
    # 选取数目特征和小波特征
#     a = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
#     feature_sample_array = feature_sample_array[:,a]
    # 选取距离特征和小波特征
#     a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
#     feature_sample_array = feature_sample_array[:,a]
    print("总样本数据维度：",feature_sample_array.shape)
    print("总样本标签维度：",label_array.shape)
    
    # 分层k折交叉验证
    skf = StratifiedKFold(n_splits=3)
    # 储存每一轮的结果
    train_score_list = []
    test_score_list = []
    feature_sample_array = preprocessing.scale(feature_sample_array)   # processing预处理
    for train, test in skf.split(feature_sample_array,label_array):
        # 准备训练集
        train_data = torch.from_numpy(feature_sample_array[train,:]).float()   # 划分好的训练集
        train_label = torch.zeros((len(label_array[train]),clf_num))
        for i in range(len(label_array[train])):
            train_label[i,label_array[i]] = 1
        train_label = train_label.float()
        test_data = torch.from_numpy(feature_sample_array[test,:]).float()  # 划分好的测试集
        test_label = torch.zeros((len(label_array[test]),clf_num))
        for i in range(len(label_array[test])):
            test_label[i,label_array[i]] = 1
        test_label = test_label.float()
        print(train_label.shape)
        print(test_label.shape)
        
        # 准备网络，优化器等
        num_epochs = 50
        lr = [1,1,1]
        batch_size = [64, 64, 64]
        class_net = [nn.Sequential(nn.Linear(20,2),nn.ReLU()),
                     nn.Sequential(nn.Linear(20,3),nn.ReLU()),
                     nn.Sequential(nn.Linear(20,4),nn.ReLU())]
        for net in class_net:
            net.apply(init_weights)
        loss = nn.CrossEntropyLoss(reduction='none')
        optimizer_class = [torch.optim.SGD(class_net[0].parameters(),lr=lr[0]),
                          torch.optim.SGD(class_net[1].parameters(),lr=lr[1]),
                          torch.optim.SGD(class_net[2].parameters(),lr=lr[2])]
        
        # 利用fc线性层训练进行分类
        train_acc_list = train_process(class_net[clf_num-2],optimizer_class[clf_num-2],loss,train_data,train_label,batch_size[clf_num-2],num_epochs)
        # 计算训练集、测试集准确率
#         predicted_train_label, train_acc = test_process(class_net[clf_num-2],train_data,train_label)
        predicted_test_label, test_acc = test_process(class_net[clf_num-2],test_data,test_label)
        
        train_score_list.append(train_acc_list[-1])
        test_score_list.append(test_acc)

    print("Clf_num:",clf_num)
    print(train_score_list)
    print(test_score_list)
    print("训练集准确率：",np.array(train_score_list).mean())
    print("测试集准确率：",np.array(test_score_list).mean())

In [17]:
clf_num = [2,3,4]
for num in clf_num:
    train_fc(num)

类别1样本数量： 511
类别2样本数量： 202
类别3样本数量： 104
类别4样本数量： 159
总样本数据维度： (722, 20)
总样本标签维度： (722,)
torch.Size([481, 2])
torch.Size([241, 2])
torch.Size([481, 2])
torch.Size([241, 2])
torch.Size([482, 2])
torch.Size([240, 2])
Clf_num: 2
[tensor(0.7505, dtype=torch.float64), tensor(0.7734, dtype=torch.float64), tensor(0.7490, dtype=torch.float64)]
[1.0, 0.8796680497925311, 1.0]
训练集准确率： 0.7576237265033946
测试集准确率： 0.9598893499308437
类别1样本数量： 511
类别2样本数量： 202
类别3样本数量： 104
类别4样本数量： 159
总样本数据维度： (477, 20)
总样本标签维度： (477,)
torch.Size([318, 3])
torch.Size([159, 3])
torch.Size([318, 3])
torch.Size([159, 3])
torch.Size([318, 3])
torch.Size([159, 3])
Clf_num: 3
[tensor(0.6604, dtype=torch.float64), tensor(0.6824, dtype=torch.float64), tensor(0.5000, dtype=torch.float64)]
[0.5974842767295597, 0.5094339622641509, 1.0]
训练集准确率： 0.6142557651991614
测试集准确率： 0.7023060796645701
类别1样本数量： 511
类别2样本数量： 202
类别3样本数量： 104
类别4样本数量： 159
总样本数据维度： (416, 20)
总样本标签维度： (416,)
torch.Size([277, 4])
torch.Size([139, 4])
torch.Size([27